<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#我们开始吧" data-toc-modified-id="我们开始吧-1">我们开始吧</a></span><ul class="toc-item"><li><span><a href="#做一些准备工作" data-toc-modified-id="做一些准备工作-1.1">做一些准备工作</a></span><ul class="toc-item"><li><span><a href="#引入库及模块" data-toc-modified-id="引入库及模块-1.1.1">引入库及模块</a></span></li><li><span><a href="#预定义变量" data-toc-modified-id="预定义变量-1.1.2">预定义变量</a></span></li></ul></li><li><span><a href="#录入你的个人信息" data-toc-modified-id="录入你的个人信息-1.2">录入你的个人信息</a></span><ul class="toc-item"><li><span><a href="#基本信息" data-toc-modified-id="基本信息-1.2.1">基本信息</a></span></li><li><span><a href="#额外信息" data-toc-modified-id="额外信息-1.2.2">额外信息</a></span></li></ul></li><li><span><a href="#对目标组合评分" data-toc-modified-id="对目标组合评分-1.3">对目标组合评分</a></span></li></ul></li><li><span><a href="#查看结果" data-toc-modified-id="查看结果-2">查看结果</a></span></li><li><span><a href="#计算细节" data-toc-modified-id="计算细节-3">计算细节</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#基本参数设置" data-toc-modified-id="基本参数设置-3.0.1">基本参数设置</a></span></li><li><span><a href="#联合分析" data-toc-modified-id="联合分析-3.0.2">联合分析</a></span></li></ul></li></ul></li></ul></div>

# 我们开始吧

这是一个简单的计算器，我们挑选了6个在选择男\女朋友的时候可能会考虑的方面，分别是**颜值**、**成绩**、**兴趣**、**年龄**、**身高**、**家庭**，通过填答一些相关的问题，最终你会看到你在这些方面的选择倾向。

- 这个小玩意儿基于交互式文档 Jupyter Notebook 制作，实际上所有的实现细节都在文档内部，在完成之后你可以查看所有细节，并就相关问题跟我们讨论。

In [339]:
# 颜值： look ["低", "一般", "高"]
# 成绩： grade ["较差", "一般", "优秀"]
# 爱好： hobby ["毫不相关", "互补", "相似"]
# 年龄： age ["更小", "同级", "更大"]
# 身高： height ["对方更矮一些", "相似（5cm以内）", "对方更高一些"]
# 家庭： family ["不那么好", "一般", "更好"]

## 做一些准备工作

引入程序包，定义必要的全局变量。

### 引入库及模块

In [340]:
# 控件库
import ipywidgets as widgets
# 显示控件的方法
from IPython.display import display
# 字典深拷贝
import copy
# 时间日期
import datetime
import time
# 数据计算包
import numpy as np
import pandas as pd
import statsmodels.api as sm

### 预定义变量

In [341]:
# @description 基于指定的 field 将 dict_list 转换为 list
# @param list 待处理的 dict_list
# @param aim_list? 目标输出 list
# @param field? 要提取的字段名
# @return list 一个新的 list
def dicts2list(dict_list, aim_list=[], field='value'):
    new_list = aim_list[:]
    for item in dict_list:
        new_list.append(item[field])
    return new_list


# @description
# @param dict 需要遍历处理的字典
# @param fn 字典字段值的处理函数，该函数第一参数为字典字段值，输出处理后的字段值
# @param fn_args 传给处理函数（fn）的参数
# @return dict 一个新的字典
def dictForEach(dict, fn, fn_args):
    new_dict = {}
    for name, item in dict.items():
        new_dict[name] = fn(item, **fn_args)
    return new_dict

In [342]:
# 预定义属性名列表
attr_list = tuple([{
    "zh": "颜值",
    "en": "look"
}, {
    "zh": "成绩",
    "en": "grade"
}, {
    "zh": "兴趣",
    "en": "hobby"
}, {
    "zh": "年龄",
    "en": "age"
},{
    "zh": "身高",
    "en": "height"
}, {
    "zh": "家庭",
    "en": "family"
}])
attr_list_en = tuple(dicts2list(attr_list, field = 'en'))
attr_list_zh = tuple(dicts2list(attr_list, field = 'zh'))

# 预定义属性及值的字典
dict = {
    "look": [{
        "name": "低",
        "value": 0
    }, {
        "name": "一般",
        "value": 1
    }, {
        "name": "高",
        "value": 2
    }],
    "grade": [{
        "name": "较差",
        "value": 0
    }, {
        "name": "一般",
        "value": 1 
    }, {
        "name": "优秀",
        "value": 2
    }],
    "hobby": [{
        "name": "毫不相关",
        "value": 0
    }, {
        "name": "互补",
        "value": 1
    }, {
        "name": "相似",
        "value": 2
    }],
    "age": [{
        "name": "更小",
        "value": 0
    }, {
        "name": "同级",
        "value": 1
    }, {
        "name": "更大",
        "value": 2
    }],
    "height": [{
        "name": "对方更矮一些",
        "value": 0
    }, {
        "name": "相似（5cm以内）",
        "value": 1
    }, {
        "name": "对方更高一些",
        "value": 2
    }],
    "family": [{
        "name": "不那么好",
        "value": 0
    }, {
        "name": "一般",
        "value": 1 
    }, {
        "name": "更好",
        "value": 2
    }]
}
# 预定义属性及值的列表
name_list_dict = dictForEach(dict, dicts2list, {"field": "name"})
value_list_dict = dictForEach(dict, dicts2list, {"field": "value"})

In [343]:
# print('预定义的属性包括：{keys}'.format(keys = dict.keys()))
# print('预定义属性及值的列表：{name_list_dict}'.format(name_list_dict = name_list_dict))

## 录入你的个人信息

### 基本信息

In [344]:
student_id = widgets.Text(value='',
                    description='学号',
                    placeholder='请输入你的学号',
                    disabled=False)
gender =  widgets.Dropdown(
    options=['男', '女'],
    value='女',
    description='性别',
    disabled=False
)
height = widgets.IntText(
    value=170,
    description='身高',
    placeholder='输入你的身高',
    disabled=False
)
education = widgets.Dropdown(
    options=['本科', '硕士', '博士'],
    value='本科',
    description='学历',
    disabled=False,
)
birthday = widgets.DatePicker(
    value=datetime.date(1998, 7, 28),
    description='生日',
    disabled=False
)

base_info = {}
def save_base_info(detail):
    # 提取字段值
    base_info["student_id"] = str(student_id.value)
    base_info["height"] = str(height.value)
    base_info["gender"] = str(gender.value)
    base_info["education"] = str(education.value)
    base_info["birthday"] = str(birthday.value)
    
    # 更新按钮状态以示提交
    detail.icon = "check"
    detail.button_style = "success"
    detail.description = "保存成功"
    print(base_info)
    
save_btn = widgets.Button(
    description='确认',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='确认信息无误并保存',
    icon=''
)    
save_btn.on_click(save_base_info)

display(student_id, height, gender, education, birthday, save_btn)

Text(value='', description='学号', placeholder='请输入你的学号')

IntText(value=170, description='身高')

Dropdown(description='性别', index=1, options=('男', '女'), value='女')

Dropdown(description='学历', options=('本科', '硕士', '博士'), value='本科')

DatePicker(value=datetime.date(1998, 7, 28), description='生日')

Button(button_style='primary', description='确认', style=ButtonStyle(), tooltip='确认信息无误并保存')

### 额外信息

In [345]:
# 容器
dropdowns = {}
# @description 根据预定义属性及其选项值设置widget，并初始化相关控制器
def generateDropdowns(cfg):
    global dropdowns
    for i in range(len(attr_list)):
        _item = dropdowns[attr_list_en[i]] = {}
        _item["initialed"] = False
        # dropdown 的选项
        _options = name_list_dict[attr_list_en[i]][:]
        _options.append(cfg["placeholder"])
        # dropdown 的描述
        _description = attr_list_zh[i]
        # 创建dropdown
        slt = widgets.Dropdown(options=_options,
                               description=_description,
                               description_tooltip='请选择…',
                               disabled=False)
        # 为 dropdown widget 实例绑定事件监听
        slt.observe(handleDropdownChange, names=['index'])
        slt.index = len(_options) - 1
        _item["widget"] = slt
        _item["options"] = _options
        _item["changed"] = False
        _item["initialed"] = True


def handleDropdownChange(detail):
    # 取到对应的英文属性名
    attr_name_zh = detail.owner.description
    attr_name_en = attr_list_en[list(attr_list_zh).index(attr_name_zh)]
    # 获取目标dropdown
    dropdown = dropdowns[attr_name_en]
    # 未初始化完成，什么都不做
    if not dropdown["initialed"]:
        return
    # 第一次更新值的时候将placeholder去掉
    if not dropdown["changed"]:
        dropdown["changed"] = True
        _index = detail["new"]
        dropdown["options"].pop()
        dropdown["widget"].options = dropdown["options"]
        dropdown["widget"].index = _index

# 生成并初始化 dropdowns
generateDropdowns({
    "placeholder": "点击进行选择…"
})

# 提取封装的widget,用于展示
dropdown_list = dicts2list(list(dropdowns.values()), field='widget')
display(*dropdown_list)

save_btn2 = widgets.Button(
    description='确认',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='确认所有字段填答无误并保存',
    icon=''
)

values = {}
def save_values(detail):
    # 确认所有字段填写完毕
    changes = dicts2list(list(dropdowns.values()), field='changed')
    if changes.count(False) >= 1 :
        detail.button_style = "warning"
        return
    
    # 提取字段值
    for attr in attr_list_en:
        values[attr] = dropdowns[attr]["widget"].value
    
    # 更新按钮状态以示提交
    detail.icon = "check"
    detail.button_style = "success"
    detail.description = "保存成功"
    print(values)

save_btn2.on_click(save_values)
display(save_btn2)

Dropdown(description='颜值', description_tooltip='请选择…', index=3, options=('低', '一般', '高', '点击进行选择…'), value='点击…

Dropdown(description='成绩', description_tooltip='请选择…', index=3, options=('较差', '一般', '优秀', '点击进行选择…'), value='…

Dropdown(description='兴趣', description_tooltip='请选择…', index=3, options=('毫不相关', '互补', '相似', '点击进行选择…'), value…

Dropdown(description='年龄', description_tooltip='请选择…', index=3, options=('更小', '同级', '更大', '点击进行选择…'), value='…

Dropdown(description='身高', description_tooltip='请选择…', index=3, options=('对方更矮一些', '相似（5cm以内）', '对方更高一些', '点击进…

Dropdown(description='家庭', description_tooltip='请选择…', index=3, options=('不那么好', '一般', '更好', '点击进行选择…'), value…

Button(button_style='primary', description='确认', style=ButtonStyle(), tooltip='确认所有字段填答无误并保存')

## 对目标组合评分

In [346]:
# 预定义属性组合
cases = [{
    "look": 1,
    "grade": 1,
    "hobby": 2,
    "age": 1,
    "height": 1,
    "family": 0
}, {
    "look": 2,
    "grade": 0,
    "hobby": 0,
    "age": 1,
    "height": 1,
    "family": 1
}, {
    "look": 0,
    "grade": 2,
    "hobby": 2,
    "age": 1,
    "height": 2,
    "family": 0
}, {
    "look": 0,
    "grade": 2,
    "hobby": 0,
    "age": 1,
    "height": 0,
    "family": 1
}, {
    "look": 1,
    "grade": 2,
    "hobby": 1,
    "age": 0,
    "height": 2,
    "family": 1
}, {
    "look": 2,
    "grade": 1,
    "hobby": 2,
    "age": 0,
    "height": 2,
    "family": 1
}, {
    "look": 2,
    "grade": 2,
    "hobby": 1,
    "age": 2,
    "height": 1,
    "family": 0
}, {
    "look": 1,
    "grade": 2,
    "hobby": 0,
    "age": 0,
    "height": 1,
    "family": 2
}, {
    "look": 2,
    "grade": 1,
    "hobby": 0,
    "age": 0,
    "height": 0,
    "family": 0
}, {
    "look": 0,
    "grade": 0,
    "hobby": 1,
    "age": 0,
    "height": 0,
    "family": 0
}, {
    "look": 1,
    "grade": 0,
    "hobby": 2,
    "age": 2,
    "height": 0,
    "family": 1
}, {
    "look": 1,
    "grade": 1,
    "hobby": 1,
    "age": 1,
    "height": 0,
    "family": 2
}, {
    "look": 0,
    "grade": 1,
    "hobby": 0,
    "age": 2,
    "height": 2,
    "family": 2
}, {
    "look": 0,
    "grade": 0,
    "hobby": 2,
    "age": 0,
    "height": 1,
    "family": 2
}, {
    "look": 2,
    "grade": 2,
    "hobby": 2,
    "age": 2,
    "height": 0,
    "family": 2
}, {
    "look": 0,
    "grade": 1,
    "hobby": 1,
    "age": 2,
    "height": 1,
    "family": 1
}, {
    "look": 2,
    "grade": 0,
    "hobby": 1,
    "age": 1,
    "height": 2,
    "family": 2
}, {
    "look": 1,
    "grade": 0,
    "hobby": 0,
    "age": 2,
    "height": 2,
    "family": 0
}]

In [347]:
tab_contents = cases[:]
cards = {"valueSliders": [], "btns": []}
score_result = []

# @description 生成保存分数的btn
def generateScoreSaveBtn():
    global score_result
    score_save_btn = widgets.Button(
        description='提交填答结果',
        disabled=False,
        button_style='primary',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='填答完毕，提交填答结果',
        icon='',
        layout=widgets.Layout(left="5%", width="90%"))
    def getCardsScore(detail):
        values = []
        for item in cards["valueSliders"]:
            values.append(item.value)
            if len(score_result) < len(tab_contents):
                score_result.append(item.value)
    score_save_btn.on_click(getCardsScore)
    return score_save_btn


# 生成保存分数的按钮
score_save_btn = generateScoreSaveBtn()


# @description 切换到下一个tab
def nextTab():
    saveBtnDisplayed = False
    def nextTabInner(detail):
        nonlocal saveBtnDisplayed
        if tab.selected_index >= len(cards["btns"]) - 1 :
            if not saveBtnDisplayed :
                saveBtnDisplayed = True
                display(score_save_btn)
        else:
            tab_index_now = tab.selected_index
            tab.selected_index = tab_index_now + 1
    return nextTabInner


# 初始化 nextTab 方法
nextTab = nextTab()


# @description 生成评分卡片
# @return contents 填充tab的内容列表
def generateContents():
    children = []
    # 遍历卡片原始数据
    for index in range(len(tab_contents)):
        _case = tab_contents[index]
        # 容器，保存呈现属性名值对的控件
        _content = []
        for attr_item, attr_value in _case.items():
            # 将原始数据组织成 属性名：属性值 的格式，并生成显示组件
            attr = widgets.HTML('{}:{}'.format(
                attr_list_zh[list(attr_list_en).index(attr_item)],
                name_list_dict[attr_item][value_list_dict[attr_item].index(
                    attr_value)]),
                                layout=widgets.Layout(width='50%',
                                                      padding='20px 0'))
            _content.append(attr)
        # 评分条
        slider = widgets.IntSlider(value=0,
                                   min=0,
                                   max=10,
                                   step=1,
                                   description='评分：',
                                   disabled=False,
                                   continuous_update=False,
                                   orientation='horizontal',
                                   readout=True,
                                   readout_format='d',
                                   layout=widgets.Layout(width='80%'))
        # 统一保存评分条实例
        cards["valueSliders"].append(slider)
        # 按钮
        next_btn = widgets.Button(
            description='下一个',
            disabled=False,
            button_style=
            'primary',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='确认并填答下一个',
            icon='',
            layout=widgets.Layout(width='20%'))
        next_btn.on_click(nextTab)
        # 统一保存按钮实例
        cards["btns"].append(next_btn)
        # 布局并作为整体保存
        children.append(
            widgets.VBox(
                [widgets.HBox(_content),
                 widgets.HBox([slider, next_btn])]))
    return children


# 生成tab并display
tab = widgets.Tab()
# tab内容
tab.children = generateContents()
# tab标题
for i in range(len(tab_contents)):
    tab.set_title(i, str(i + 1))
display(tab)

# 查看结果

In [348]:
result_displayed = False
def calcResult(detail):
    global result_displayed
    if result_displayed: return
    global score_result
    # 设置正交试验设计中每个样本的变量情况
    ## 所有水平列表
    LEVEL_LIST = ["A", "B", "C", "D", "E", "F"]
    LEVEL_NAME = ["颜值", "成绩", "爱好", "年龄", "身高", "家庭"]
    ## 所有因素列表
    FACTORY_LIST = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3',
                    'C1', 'C2', 'C3', 'D1', 'D2', 'D3',
                    'E1', 'E2', 'E3', 'F1', 'F2', 'F3']
    FACTORY_NAME = ["高", "一般", "低", "优秀", "一般", "较差",
                    "相似", "互补", "毫不相关", "更大", "同级", "更小",
                    "身高", "相似", "更低", "更好", "一般", "不那么好"]
    FACTORY_LEVEL_NAME = ["颜值", "颜值", "颜值", "成绩", "成绩", "成绩",
                          "爱好", "爱好", "爱好", "年龄", "年龄", "年龄",
                          "身高", "身高", "身高", "家庭", "家庭", "家庭"]
    ## 正交试验设计结构
    CASE_STRUCTURE = ['A2B2C1D2E2F3', 'A1B3C3D2E2F2', 'A3B1C1D2E1F3', 'A3B1C3D2E3F2', 'A2B1C2D3E1F2', 'A1B2C1D3E1F2',
                      'A1B1C2D1E2F3', 'A2B1C3D3E2F1', 'A1B2C3D3E3F3', 'A3B3C2D3E3F3', 'A2B3C1D1E3F2', 'A2B2C2D2E3F1',
                      'A3B2C3D1E1F1', 'A3B3C1D3E2F1', 'A1B1C1D1E3F1', 'A3B2C2D1E2F2', 'A1B3C2D2E1F1', 'A2B3C3D1E1F3']
    MARK = score_result
    
    # 生成初始化个案表
    columns_list = ["Mark"]
    columns_list.extend(FACTORY_LIST)
    ConjointDummyDF = pd.DataFrame(np.zeros((len(CASE_STRUCTURE), len(FACTORY_LIST) + 1)), columns=columns_list)
    
    # 生成个案得分对应表
    markDF = pd.DataFrame({"Level": CASE_STRUCTURE, "Mark": MARK})
    
    # 将正交设计分布写入个案表
    for index, row in markDF.iterrows():
        factory = []
        for i in range(0, len(LEVEL_LIST)):
            factory.append(markDF["Level"].loc[index][i * 2:i * 2 + 2])
        ConjointDummyDF.loc[index, factory] = 1
    
    
    # 将得分写入个案表
    ConjointDummyDF["Mark"] = markDF["Mark"]
    
    # 回归计算
    ## 计算回归分析的X和Y
    X = sm.add_constant(ConjointDummyDF[FACTORY_LIST])
    Y = ConjointDummyDF["Mark"]
    ## 执行回归分析
    linearRegression = sm.OLS(Y, X).fit()
    
    
    # 计算效用估计表
    ## 生成按组存储的回归系数
    level_utilities = []
    for level in LEVEL_LIST:
        temp_utility = []
        for factory in FACTORY_LIST:
            if factory[0] == level:
                temp_utility.append(linearRegression.params[factory])
        level_utilities.append(temp_utility)
    ## 计算各水平的重要性(回归系数极差)
    importance = []
    for item in level_utilities:
        importance.append(max(item) - min(item))
    ## 计算各水平的重要性百分比
    relative_importance = []
    for item in importance:
        relative_importance.append(100 * round(item / sum(importance), 3))
    ## 计算各因素的平均得分
    meanMark = []
    for i in ConjointDummyDF.columns[1:]:
        newMeanMark = ConjointDummyDF["Mark"].loc[ConjointDummyDF[i] == 1].mean()
        meanMark.append(newMeanMark)
    ## 计算各因素平均得分
    totalMeanMark = sum(meanMark) / len(meanMark)
    ## 计算各因素效用值
    utility = []
    for i in range(len(meanMark)):
        name = sorted(FACTORY_LIST)[i]
        utility.append(meanMark[i] - totalMeanMark)
    
    # 结果呈现
    ## 效用值表
    tableUtilities = pd.DataFrame({"水平": FACTORY_LEVEL_NAME, "因素": FACTORY_NAME, "效用值": utility})
    display(tableUtilities)
    ## 水平重要性表
    tableImportance = pd.DataFrame({"水平": LEVEL_NAME, "重要性": relative_importance})
    display(tableImportance)
    
    result_displayed = True

In [349]:
# @description 生成查看结果的btn
def generateResultBtn():
    global score_result,calcResult
    result_btn = widgets.Button(
        description='看计算结果',
        disabled=False,
        button_style='primary',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='填答完毕，提交填答结果',
        icon='',
        layout=widgets.Layout(left="5%", width="90%"))
    result_btn.on_click(calcResult)
    return result_btn

# 生成保存分数的按钮
result_btn = generateResultBtn()
result_btn

Button(button_style='primary', description='看计算结果', layout=Layout(left='5%', width='90%'), style=ButtonStyle()…

# 计算细节

### 基本参数设置

设置正交试验设计中每个样本的变量情况

In [350]:
# 所有水平列表
LEVEL_LIST = ["A", "B", "C", "D", "E", "F"]
LEVEL_NAME = ["颜值", "成绩", "爱好", "年龄", "身高", "家庭"]
# 所有因素列表
FACTORY_LIST = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3',
                'C1', 'C2', 'C3', 'D1', 'D2', 'D3',
                'E1', 'E2', 'E3', 'F1', 'F2', 'F3']
FACTORY_NAME = ["高", "一般", "低", "优秀", "一般", "较差",
                "相似", "互补", "毫不相关", "更大", "同级", "更小",
                "身高", "相似", "更低", "更好", "一般", "不那么好"]
FACTORY_LEVEL_NAME = ["颜值", "颜值", "颜值", "成绩", "成绩", "成绩",
                      "爱好", "爱好", "爱好", "年龄", "年龄", "年龄",
                      "身高", "身高", "身高", "家庭", "家庭", "家庭"]
# 正交试验设计结构
CASE_STRUCTURE = ['A2B2C1D2E2F3', 'A1B3C3D2E2F2', 'A3B1C1D2E1F3', 'A3B1C3D2E3F2', 'A2B1C2D3E1F2', 'A1B2C1D3E1F2',
                  'A1B1C2D1E2F3', 'A2B1C3D3E2F1', 'A1B2C3D3E3F3', 'A3B3C2D3E3F3', 'A2B3C1D1E3F2', 'A2B2C2D2E3F1',
                  'A3B2C3D1E1F1', 'A3B3C1D3E2F1', 'A1B1C1D1E3F1', 'A3B2C2D1E2F2', 'A1B3C2D2E1F1', 'A2B3C3D1E1F3']

给定正交试验样本的得分情况

In [351]:
MARK = [3, 4, 8, 2, 1, 2, 4, 7, 6, 4, 7, 6, 2, 7, 6, 10, 3, 9]

### 联合分析

1.导入包

In [352]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

2.生成初始化个案表

In [353]:
columns_list = ["Mark"]
columns_list.extend(FACTORY_LIST)
ConjointDummyDF = pd.DataFrame(np.zeros((len(CASE_STRUCTURE), len(FACTORY_LIST) + 1)), columns=columns_list)

3.生成个案得分对应表

In [354]:
markDF = pd.DataFrame({"Level": CASE_STRUCTURE, "Mark": MARK})

4.将正交设计分布写入个案表

In [355]:
for index, row in markDF.iterrows():
    factory = []
    for i in range(0, len(LEVEL_LIST)):
        factory.append(markDF["Level"].loc[index][i * 2:i * 2 + 2])
    ConjointDummyDF.loc[index, factory] = 1

5.将得分写入个案表

In [356]:
ConjointDummyDF["Mark"] = markDF["Mark"]

6.回归计算

In [357]:
# 计算回归分析的X和Y
X = sm.add_constant(ConjointDummyDF[FACTORY_LIST])
Y = ConjointDummyDF["Mark"]
# 执行回归分析
linearRegression = sm.OLS(Y, X).fit()

7.计算效用估计表

In [358]:
# 生成按组存储的回归系数
level_utilities = []
for level in LEVEL_LIST:
    temp_utility = []
    for factory in FACTORY_LIST:
        if factory[0] == level:
            temp_utility.append(linearRegression.params[factory])
    level_utilities.append(temp_utility)

In [359]:
# 计算各水平的重要性(回归系数极差)
importance = []
for item in level_utilities:
    importance.append(max(item) - min(item))

In [360]:
# 计算各水平的重要性百分比
relative_importance = []
for item in importance:
    relative_importance.append(100 * round(item / sum(importance), 3))

In [361]:
# 计算各因素的平均得分
meanMark = []
for i in ConjointDummyDF.columns[1:]:
    newMeanMark = ConjointDummyDF["Mark"].loc[ConjointDummyDF[i] == 1].mean()
    meanMark.append(newMeanMark)

In [362]:
# 计算各因素平均得分
totalMeanMark = sum(meanMark) / len(meanMark)

In [363]:
# 计算各因素效用值
utility = []
for i in range(len(meanMark)):
    name = sorted(FACTORY_LIST)[i]
    utility.append(meanMark[i] - totalMeanMark)

In [364]:
tableUtilities = pd.DataFrame({"水平": FACTORY_LEVEL_NAME, "因素": FACTORY_NAME, "效用值": utility})
tableUtilities

,水平,因素,效用值
0,颜值,高,-0.888889
1,颜值,一般,0.444444
2,颜值,低,0.444444
3,成绩,优秀,-0.388889
4,成绩,一般,-0.222222
5,成绩,较差,0.611111
6,爱好,相似,0.444444
7,爱好,互补,-0.388889
8,爱好,毫不相关,-0.055556
9,年龄,更大,1.277778


8.生成水平重要性表

In [365]:
tableImportance = pd.DataFrame({"水平": LEVEL_NAME, "重要性": relative_importance})
tableImportance

,水平,重要性
0,颜值,16.3
1,成绩,12.2
2,爱好,10.2
3,年龄,24.5
4,身高,20.4
5,家庭,16.3
